<a href="https://colab.research.google.com/github/davidnoone/PHYS332_FluidExamples/blob/main/03_ShallowWaterWave.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shallow water equation in 1d
Consider the a shallow water body with the surface as $z = H + \eta$, where $H$ is the (constant) mean depth relative to a reference height at z=0. We may altunatievly write surface pertubation in terms of the depth of the water column as $\eta = h + \eta_b$, where the bottom topogarph is known.

Mass continuity in 1d requires:

$$
\frac{\partial h}{\partial t} + \frac{\partial (hu)}{\partial x}  = 0 
                                                                \tag{1}
$$

Where u is the depth integrated (mean) velocity, and it is goverened by the (vertically integrated) momentum equation. 

$$
\frac{\partial u}{\partial t} = -u\frac{\partial u}{\partial x}
   -g \frac{\partial h}{\partial x} 
                                                                \tag{2}
$$

These non linear equations describe the shallow water system, which supports a wide range to different behviours. 




## Linearized case

We write the quantities h and u as a mean and deviation from the mean. $h = H + h'$ and $u' = u$, where we assume the mean velocity $U=0$. Substituting into equation 1 and 2, 

$$
\frac{\partial h'}{\partial t} + H \frac{\partial u}{\partial x}  = 0 
                                                                \tag{3}
$$
and
$$
\frac{\partial u}{\partial t} +  g \frac{\partial h'}{\partial x} = 0
                                                                 \tag{4}
$$

Here, we have used the definitions that $\eta' = h' + \eta_b$ and noting that we assume $h' << H$ such that $h' + H \approx H$. We can make some immediate simplifications. Further with the fluid velocity u' defined as being small, then terms involving products such as $u'u'$ can be neglected (i.e., linearlized) .

Taking $\partial/\partial t$ of (3) and $\partial/\partial x$ (4) reaveals a common cross term ($\partial^2 u/\partial x\partial t$) which may be elliminated to combine the two first order equation into a single second order equation:

$$
\frac{\partial^2 h'}{\partial t^2} 
        + c^2 \frac{\partial^2 h'    }{\partial x^2} = 0
                                                      \tag{5}
$$

where $c^2 = gH$. Equation (5) takes the form of a wave equation with a wave phase speed given by c. For this linearized case, the phase speed depends only on the mean water depth, H. 


##A numerical solution

We may immediately write down a finite difference analog for equation (5). 

$$
\frac{h_i^{n-1} - 2h_i^{n} + h_i^{n+1}}{(2\Delta t)^2} = 
    c^2 \frac{h_{i-1}^{n} - 2 h_{i}^{n} + h_{i+1}^{n}}{(2\Delta x)^2}
$$

Where the prime has been dropped for clarity. Writing the "Courant number" as 
$$
r = c \frac{\Delta t}{\Delta x} 
$$

we may rearrange to give a prediction equation (i.e., terms  at the future time n+1 on the left, and terms involving the past on the right)
$$
   h_i^{n+1} = (2 h_i^{n} - h_i^{n-1}) +
     r^2 (h_{i-1}^{n} - 2 hi_{i}^{n} + h_{i+1}^{n})
$$

The first term on the right can be viewed as a first order extrapolation to time $n+1$, with the remainder like a correction that provides an update based on the gradient at the centered time $n$. Some algebra let's us write a final form:

$$
   h_i^{n+1} = 2(1-r^2) h_i^{n} - h_i^{n-1} 
               + r^2 (h_{i-1}^{n} + h_{i+1}^{n})
                                                  \tag{8}
$$


This system is stable for $r < 1$.

This can be integrated by selecting a computational grid with spacing $\Delta x$ and marching forward in time by small increments given by the time step $\Delta t$. Integration is continued until the needed forecast is produced. 

In [ ]:
from matplotlib import rc
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
rc('animation', html='jshtml')

print("Imported all the modules")

In [ ]:
# Model runtime settings

tmax  = 10                          # Run duation [seconds]

# Space domain
xmin = -5.0                         # Spatial domain maximum [meters]
xmax =  5.0                         # Spatial domain [meters]
nx   = 101                          # number of points
dx = (xmax - xmin)/(nx-1)           # using nx-1 means the ends are included

xpts = np.linspace(xmin,xmax,nx)    # Values of x 

hbar = 1.0                          # Set reference depth "H" [metres]
gravity = 1.0                       # Define gravity (g=9.81 m/s2 on earth)

csq    = hbar*gravity               # phase speed csq = g*H for shallow water [m/s2]
cno_setting = 0.5                   # This must be < 1. Use 0.5 for safety!

dtime = cno_setting*dx/np.sqrt(csq)
nsteps = int(tmax/dtime) + 1        # number of steps needed to get to tmax

print('nx=',nx,'dtime = ',dtime, ' which means nsteps=',nsteps)

nskip  = 5                          # steps between output frames 

rsq = (dtime/dx)**2
rcsq = rsq*csq
twomr2 = 2*(1-rcsq)

# Set up indices for the "i+1" and "i-1". Handy later on.
i = np.arange(nx,dtype='int')
ip = i+1
im = i-1

# Set reflective boundary conditions
ip[nx-1] = nx-2
im[   0] = 1

# This version for periodic boundary conditions
#ip[nx-1] = 0
#im[   0] = nx-1

# Create the main state array
h = np.zeros((nsteps,nx))

# Initial conditions - set as a Gaussian "mountain" of water
print("initial conditions")
x0 = 0.0                # x location of mountain
xw = 0.5                # width in units of "x"
h[0,:] = np.exp( -((xpts-x0)**2)/(xw*xw))   # Initial conditions, "gaussian mountain"
h[1,:] = h[0,:]             # First step is explicit forward.

# Do the forecast! Integrate equation (8) over little steps "dt"
print("Integrating: nsteps=",nsteps)
for n in range(1,nsteps-1):
    h[n+1,i] = twomr2*h[n,i] - h[n-1,i] + rcsq*(h[n,im] + h[n,ip]) 

print('Integration complete')

In [ ]:
# Plot first few frames. Is it working?
plt.plot(xpts,h[0,:])
plt.plot(xpts,h[3*nskip,:])
plt.plot(xpts,h[6*nskip,:])


## Visualizing the result

You should see your initial "mountain" of water, that spreads outwards like ripples on a pond. The wave travels both directions (like ripples in a pond).


### More output?
The output has been stored in a single array, h, with dimensions (nsteps, nx) . We can work over the array to generate plots for each time to make an animation. If you have many steps, then this may become slow. The code is set up to "skip" some number of output times to speed it up (see nskip above).

In [ ]:
# Create an animation object by lopping over frame by frame
def plot_animation():

    # initialization function: plot the background of each frame
    def init_frame():
        line.set_data([], [])
        return line,

    # animation function.  This is called sequentially
    def plot_frame(i):
        line.set_data(xpts, h[i*nskip,:])
        ax.set(title='Shallow water wave, h = h (x,t='+str(i).zfill(4)+')')
        return line,

    # First set up the figure, the axis, and the plot elements we want to animate
    fig = plt.figure()
    ax = plt.axes(xlim=(xmin, xmax),ylim=[-0.1,1.1])
    ax.set(xlabel='X position [m]', ylabel='Wave height [m]')
    line, = ax.plot([], [], lw=2)

    nframes = int(nsteps/nskip)+1
    anim = animation.FuncAnimation(fig, plot_frame, init_func=init_frame, frames=nframes, 
                                interval=20,blit=True,repeat=True)
    plt.close()
    return anim


In [ ]:
# Generate an animation of the solution $h = h(x,t)$.
print("Rendering animation, please wait...")
plot_animation()


##Variable depth

The phase speed depends on depth $c^2 = gH$, and the water depth may not be constant. i.e., $H = H(x)$. While it is possible to account for this within the original (non-linear) equatons, we can also account for it in the linear equations, by considering the problem of a wave equation with the variable coefficient. 

$$
   \frac{\partial^2 h}{\partial t^2} = \
      g \frac{\partial }{\partial x} \left(
          H(x) \frac{\partial h}{\partial x} 
        \right)
                        \tag{9}
$$


Following the example above, we may immediately write down the finite difference analog for this equation. However, care is needed to evaluate the term in brackets at the same location on the computational grid. Specifically, write the $F = H \partial h/\partial x$, and note that this quantity that needs to be evaluated at the same grid location. This is most naturally done at "half points", denoted by local $i+1/2$, etc. In general, H might be some analytic function, or come from some external data source, and it can be specified at these locations. 

Let us write this term in finite difference form as
$$ 
  F_{i+1/2} = H_{i-1/2} \left( \frac{h_{i+1} - h_{i}}{\Delta x} \right)
$$


and then our wave equation becomes


$$
\frac{h_i^{n-1} - 2h_i^{n} + h_i^{n+1}}{(2\Delta t)^2} = 
    g \frac{F_{i+1/2} -  F_{i-1/2}} {\Delta x}
$$

Substituting in the definion of F, the finite difference analog of equation (9) is:

$$
\frac{h_i^{n-1} - 2h_i^{n} + h_i^{n+1}}{(2\Delta t)^2} = 
    \frac{g}{(\Delta x)^2} \left[
            H_{i+1/2} ( h_{i+1}^n - h_{i}^n ) 
         -  H_{i-1/2} ( h_{i}^n - h_{i-1}^n )
                       \right]
$$



Finally, we may rearrange to give a prediction equation
$$
   h_i^{n+1} = (2 h_i^{n} - h_i^{n-1}) +
     g r^2 \left[
            H_{i+1/2} ( h_{i+1}^n - h_{i}^n ) 
         -  H_{i-1/2} ( h_{i}^n - h_{i-1}^n )
                       \right]
$$

Notice a small detail that for a grid with $N$ cells that have midpoints at locations $i$, but there are $N+1$ "interfaces" between cells that are denoted by the "half" locations. Therefore the function H must be asisgned the $N+1$ values.

Again, we may make the forecast we need by integrating over a number of small time steps. In this case, we must forst select a topography ptofile.

Notice, that with a large H, the wave speed is fast, which can mean the Courant number r becomes large. The numerical solution is ONLY stable if r < 1. If you choose a large H, you will need to make $\Delta t$ smaller to ensure stablity.
(Or, make $\Delta x$ larger!) The code here does not automatically check for this.




In [ ]:
# Set up a topography profile.
hamp = 0.0                           # "Amplitude" of slope <= hbar [CHANGE ME]
Hbot = hbar + np.linspace(-hamp,hamp,nx+1)    # [Choose some function]

# Run the model again!
print("Integrating: nsteps=",nsteps)
for n in range(1,nsteps-1):
    h[n+1,i] = 2*h[n,i] - h[n-1,i] + \
              gravity*rsq*(Hbot[ip]*(h[n,ip]-h[n,i]) - Hbot[i]*(h[n,i]-h[n,im]))

print("Rendering animation, please wait...")
plot_animation()


# Questions

1. Using the constant depth model, confirm that the wave speed is dependent on the ocean depth, but it does not depend on the wave amplitude of "width".

2. Configure the model to represent a tsunami. Consider a 1 meter high wave 1000 km out to sea. How long does it take to reach the shore. The typical depth of the South Atlantic near New Zealand is 1.5 km. _Note: As written above in mostly non-dimensional form, we have gravity = 1! This is not the case on earth. You will need to select relevant values for gravity, xmin, xmax, and H. This may place limits on the balance of $\Delta x$ and $\Delta t$ that you will need. Choose carefully._

3. Change the bootom topography to have a "slope" as it approaches the shore (on the left). Confirm that the wave will reach the shore more 
slowly that the speed of the wave out at sea. Consider the ocean to have a slope with an amplitude relative to the mean depth.

4. What is the difference in the ampltude of the wave when it reaches the shore? 

5. ***Tsunami!*** What is the wave height when the depth gets only 1 meter deep? (What happens if Hbot = 0 @ x = xmin?)



# Outcome

This module has introduces the wave equation, and numerical solutions for time evolving flows. The concept of the Courant number and limits on this for numerical problems is a fundemental limitation on selection of spatial (x) resolution for particular time steps. Higher spatial resolution requires smaller time steps, and this is limited by the computation power at hand. This partly means your patience to wait for a high resolution solution, but also the capcity in of the computer to store the reults! Notice that $nx \times ntime$ is the number of numbers stored in the "h" array. And each number is 8 bytes. Wile not a big deal in 1d, in 3d, $nx \times ny \times nz \times ntime\times 8 $ bytes can add up quite quickly!

The module has also shown that the "small wave" approximation can be violated. As the amplitude grows with small wave speed (and small depth), we must at some point reconsider the linear assumtion, and the fact that we used small amplitude assumtion. At some point, we need to return to the linear equations, and more complex solution method. 











##Bonus: Better Tsunami model?

For the pythonically adventurous, the example above can easily be extended to 2d. One needs to define a "y" dimension, and the equations extended. Take note that the plotting now needs to be done in 2d. You may wish to use the contour (or contourf) function rather than "plot" to get a colored map in the X-Y plane. In the X-Y plane, you also need to consider the bottom topography. Some simple mathematical functions can be used, or one could consider obtaining real bathymetry data for the ocean sea floor. 

This starts to resemble and opperational tsunami forecast modelling!

## Bonus 2: Going further, why stay linear? 

One can consider solving the full non-linear equations. While still quite straight foreward, it is a somewhat more complicated problem requiring more care in representing the non-linear advection term (which we have cleverly avoided in the derivation above!)

